# Steps
1.) Set up box containing the ideal gas

2.) Create the bounds for the segments of the box used to measure the density of the particles

3.) Run the simulation and record the average density of the segments in the box

4.) Apply gravity to the simulation

5.) Run the simulation and find the new densities 


# Bonus steps
If completed, these would offer cool new things to add to the box to see their interactions 

1.) Add an object to the box to represent a large natural structure such as a mountain and measure
the effects it has on the density of the gas

2.) add another gas of different mass, initial velocity, and/or temperature and see how that effects density

# Challenges
Accurately demonstrate the change in density with altitude in such a small model

validating the model (similar to the first challenge) How do i know my model is correct?


# Import code from our ideal gas histogram model

In [41]:
from vpython import *
import numpy as np
import random as rand
import matplotlib.pyplot as plt

In [42]:
def check_collisions():
    """Determine all pairs of colliding spheres in the global particles[] list. 
        
    """
    collided_pairs = [] #a list of pairs of spheres that are overlapping
    
    for i in range(N):
        for j in range(i+1,N): #start at i+1 to avoid double counting and to avoid self-collision
            d = mag(particles[i].pos - particles[j].pos)
            if d < 2*R:
                collided_pairs.append([i,j])            
    
    return collided_pairs

In [43]:
def calc_collision(i,j):
    """Calculate new velocity, momentum, and position of pairs of colliding spheres in the global particles[] list. 

    Keyword arguments:
    i -- index of one colliding sphere
    j -- index of second colliding sphere

    """

    global particles
    
    r1 = particles[i].pos
    r2 = particles[j].pos
    v1 = particles[i].v
    v2 = particles[j].v
    p1 = m*v1
    p2 = m*v2
    v1i = v1
    v2i = v2
    
    #transform to reference frame of particle 1
    v2rel = v2 - v1
    r2rel = r2 - r1
    
    #find time when collision ocurred
    dx = dot(r2rel, hat(v2rel))
    dy = mag(cross(r2rel, hat(v2rel)))
    alpha = asin(dy/2/R)
    d = 2*R*cos(alpha) + dx
    deltat = d/mag(v2rel)
    
    #move particles backward in time to their locations at the collision
    particles[i].pos = particles[i].pos - particles[i].v*deltat
    particles[j].pos = particles[j].pos - particles[j].v*deltat
    
    #calculate new momenta using CM reference frame
    r1 = particles[i].pos
    r2 = particles[j].pos
    M = 2*m #total mass
    ptot = p1+p2 #total momentum
    vcm = ptot/M
    v1rel = v1 - vcm
    v2rel = v2 - vcm
    p1rel = m*v1rel
    p2rel = m*v2rel
    r = r2 - r1
    p1rel = p1rel - 2*dot(p1rel,hat(r))*hat(r)
    p2rel = p2rel - 2*dot(p2rel,hat(r))*hat(r)
    v1rel = p1rel/m
    v2rel = p2rel/m
    v1 = vcm + v1rel
    v2 = vcm + v2rel
    p1 = m*v1
    p2 = m*v2
    particles[i].v = v1
    particles[j].v = v2

    #move particles forward in time to their locations at the end of the time step
    particles[i].pos = particles[i].pos + particles[i].v*deltat
    particles[j].pos = particles[j].pos + particles[j].v*deltat
    





# Lets first create a scene with an ideal gas in a box unaffected by gravity and measure the density.

We will measure density by splitting the box into three imaginary segments.
Each segment is the same length and width as the box and one third of its height.

The first segment will start at the top of the box and go 1/3rd of the way down.

The second segment starts at the bottom of the first and goes down another 1/3rd.

The last segment will start below the second and will contain the remaining 1/3rd of the box at the bottom.

The segments will be more useful later when gravity is applied in order to visualize it's effects.

In [ ]:
scene = canvas(title="N Particles in a Box")

#constants
m = 1.7e-27 #mass of atom in kg
R = 0.5e-10 #radius of atom in m
L = 60*R #length of box in m
thick = L/100 #thickness of box wall in m
k = 1.4e-23 #boltzmann constant
T = 300 #temp in K
s =  np.sqrt(2*3/2*k*T/m) #initial speed
Volume = 4/3*pi*R**3 # volume of particle| expected value is 5.23598e-31
segmentVolume = L*L*(L*.3333)  #volume of each segment

print("Volume of particle(expected value:5.23598e-31)",Volume)  #check that values are expected
print("Volume of each segment",segmentVolume)

#visual objects
Lwall = box(pos = vec(-L/2, 0, 0), size = vec(thick, L, L), color=color.white)
Rwall = box(pos = vec(L/2, 0, 0), size = vec(thick, L, L), color=color.white)
Bwall = box(pos = vec(0, -L/2, 0), size = vec(L, thick, L), color=color.white)
Twall = box(pos = vec(0, L/2, 0), size = vec(L, thick, L), color=color.white)
Zwall = box(pos = vec(0, 0, -L/2), size = vec(L, L, thick), color=color.white)

particles = [] #list of particles

N=100 #number of particles

#create particles
for i in range(N):
    if i==0:
        particle = sphere(pos = L/2*vec(rand.uniform(-0.9,0.9),rand.uniform(-0.9,0.9),rand.uniform(-0.9,0.9)), radius = R, color = color.cyan, make_trail=True, retain=20, trail_radius=0.3*R)
    else:
        particle = sphere(pos = L/2*vec(rand.uniform(-0.9,0.9),rand.uniform(-0.9,0.9),rand.uniform(-0.9,0.9)), radius = R, color = color.red)

    #initial velocity
    particle.v = s*hat(vec(rand.uniform(-1,1), rand.uniform(-1,1), rand.uniform(-1,1)))
    particle.p = m*particle.v

    particles.append(particle)

#check for collisions and rerandomize positions of particles that are collided
collided_pairs = check_collisions()
while len(collided_pairs)!=0:
    for pair in collided_pairs:
        particles[pair[0]].pos = L/2*vec(rand.uniform(-1,1),rand.uniform(-1,1),rand.uniform(-1,1))
        particles[pair[1]].pos = L/2*vec(rand.uniform(-1,1),rand.uniform(-1,1),rand.uniform(-1,1))
    collided_pairs = check_collisions()

#time
t = 0
dt = R/s/10


#segment boundaries and starting amounts of particles in each boundary
topBottomBound = L/2 - L*.3333
bottomUpperBound = -L/2 + L*.3333
topAmount = 0
midAmount = 0
botAmount = 0

#Counting particles in each segment
for particle in particles:
    if particle.pos.y > topBottomBound:  #checks if the y pos of the particle is located in the upper segment 
        topAmount +=1
        
    elif particle.pos.y < bottomUpperBound:  #checks if the y pos of the particle is located in the lower segment 
        botAmount +=1
        
    else:
        midAmount +=1
        
#find out starting densities
#not right yet
topDensity = (topAmount*Volume)/segmentVolume
midDensity = (midAmount*m)/segmentVolume
botDensity = (botAmount*m)/segmentVolume


print("Starting top segment Density",topDensity)
print("Starting mid segment Density",midDensity)
print("Starting bot segment Density",botDensity)

print("top segemnt amount of particles",topAmount)
print("mid segment amount of particles",midAmount)
print("bot segment amount of particles",botAmount)


#evolution
scene.pause()

Nsteps = 1e4 #number of time steps for the loop
while t < Nsteps*dt:
    rate(1000) #sets number of loops per second in order to slow down or speed up visualization
    
    #reset the counts
    topAmount = 0
    midAmount = 0
    botAmount = 0
    
    #update position of each particle
    for particle in particles:
        particle.pos = particle.pos + particle.v*dt
        
    #handle collisions of particles with each other
    collided_pairs = check_collisions()
    for pair in collided_pairs:
        i = pair[0]
        j = pair[1]
        calc_collision(i,j)
    
    #handle collision with walls
    for particle in particles:
        if particle.pos.x > L/2:
            particle.v.x = - abs(particle.v.x)
            particle.p = m*particle.v
        elif particle.pos.x < -L/2:
            particle.v.x = abs(particle.v.x)
            particle.p = m*particle.v

        if particle.pos.y > L/2:
            particle.v.y = - abs(particle.v.y)
            particle.p = m*particle.v
        elif particle.pos.y < -L/2:
            particle.v.y = abs(particle.v.y)
            particle.p = m*particle.v

        if particle.pos.z > L/2:
            particle.v.z = - abs(particle.v.z)
            particle.p = m*particle.v
        elif particle.pos.z < -L/2:
            particle.v.z = abs(particle.v.z)
            particle.p = m*particle.v
            
            
    #Counting particles in each segment
    for particle in particles:
        if particle.pos.y > topBottomBound:  #checks if the y pos of the particle is located in the upper segment 
            topAmount +=1
        
        elif particle.pos.y < bottomUpperBound:  #checks if the y pos of the particle is located in the lower segment 
            botAmount +=1
        
        else:
            midAmount +=1
            
#     print("top Amount",topAmount)
#     print("mid Amount",midAmount)
#     print("bot Amount",botAmount)       
    t = t + dt


<IPython.core.display.Javascript object>

Volume of particle(expected value:5.23598e-31) 5.235987755982989e-31
Volume of each segment 8.9991e-27
Starting top segment Density 0.0023855218632452416
Starting mid segment Density 5.856141169672523
Starting bot segment Density 5.2894178306719555
top segemnt amount of particles 41
mid segment amount of particles 31
bot segment amount of particles 28
